In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import BatchNormalization
from flask import Flask , render_template , request , send_from_directory

In [2]:
CATEGORIES=[]
path = 'PokemonData/'
allFileList = os.listdir(path)
for file in allFileList:
    if os.path.isdir(os.path.join(path,file)):
        CATEGORIES.append(file)

In [3]:
PokemonName = CATEGORIES
num_category=len(CATEGORIES)
IMG_SIZE=64

In [4]:
vgg_model = VGG16(weights='imagenet', include_top=False,
                  input_shape=(IMG_SIZE, IMG_SIZE, 3))

x = Flatten()(vgg_model.output)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
outputs = Dense(num_category, activation='softmax')(x)

model = Model(inputs=vgg_model.inputs, outputs=outputs)

In [5]:
#存取最佳權重，並使用EarlyStopping來避免overfitting
vgg_model.trainable = True

trainable_layer = 3
for layer in vgg_model.layers[:-trainable_layer]:
    layer.trainable = False

for layer in model.layers:
    print(layer, layer.trainable)

learning_rate = 1e-4
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

<keras.engine.input_layer.InputLayer object at 0x000001E2510EF400> False
<keras.layers.convolutional.Conv2D object at 0x000001E251123970> False
<keras.layers.convolutional.Conv2D object at 0x000001E2511D1040> False
<keras.layers.pooling.MaxPooling2D object at 0x000001E25131B490> False
<keras.layers.convolutional.Conv2D object at 0x000001E2511D1610> False
<keras.layers.convolutional.Conv2D object at 0x000001E2513362B0> False
<keras.layers.pooling.MaxPooling2D object at 0x000001E25133A1C0> False
<keras.layers.convolutional.Conv2D object at 0x000001E25132D880> False
<keras.layers.convolutional.Conv2D object at 0x000001E251388040> False
<keras.layers.convolutional.Conv2D object at 0x000001E251388C10> False
<keras.layers.pooling.MaxPooling2D object at 0x000001E25138CB20> False
<keras.layers.convolutional.Conv2D object at 0x000001E25138C4F0> False
<keras.layers.convolutional.Conv2D object at 0x000001E251392D90> False
<keras.layers.convolutional.Conv2D object at 0x000001E25138CA30> False
<ker

In [6]:
model.load_weights('basic_model-best-model.h5')

In [7]:
COUNT=0
app = Flask('_name_') #呼叫Flask Function
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 1 #暫存

In [8]:
@app.route('/',endpoint="main")
def main():
    return render_template('index.html')

In [9]:
@app.route('/home',methods=['POST'])
def home():
    ratio = []
    global COUNT
    global PokemonName
    img = request.files['image']##抓圖片
    
    img.save("{}.jpg".format(COUNT))##存圖片
    img_arr = cv2.imread("{}.jpg".format(COUNT),1)##0為黑白,1為彩色
    
    img_arr = cv2.resize(img_arr,(64,64))#重整大小
    img_arr = img_arr/255.0 ##一般化
    img_arr = img_arr.reshape(1,64,64,3)
    prediction = model.predict(img_arr) ##利用模型預測
   
    
    for i in range(num_category):
        ratio.append(round(prediction[0,i],2))
   
    
    preds = np.array(ratio) 
    
    data = {'Probability' : preds, 
            'Name' : PokemonName}
    COUNT+=1 ##下一個圖片
    return render_template('prediction.html', Data = data)

In [10]:
@app.route('/load_img')
def load_img():
    global COUNT
    return send_from_directory(".","{}.jpg".format(COUNT-1))

In [ ]:
if __name__=='__main__':
    app.run(debug=False)

 * Serving Flask app "_name_" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2022 00:41:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:25] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:32] "POST /home HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:32] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:32] "GET /load_img HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:43] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:47] "POST /home HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:47] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:47] "GET /load_img HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 00:41:55] "GET /static/style.css HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2022 00:41:59] "POST 